In [ ]:
from serpapi import GoogleSearch
from langchain.tools import tool

In [ ]:
def flight_info_extract(flight_option, departure_id: str, arrival_id: str, travel_date: str):
    all_options= []
    # other_flights = results['other_flights']
    if not flight_option:
        result = f"There are no Flights from: {departure_id} to {arrival_id} on date: {travel_date}"
        return all_options, result
    for idx, flight in enumerate(flight_option):
        price = flight.get("price", -1)
        if price >= 0:
            all_options.append(
                {
                    "Flight Option": idx+1,
                    "price": price,
                    "Layover": [layover_name.get('name', "") for layover_name in flight.get('layovers', [])] if flight.get('layovers') else ["No Layover"],
                    "Total Flight Duration (hours)": round(flight.get('total_duration', 0)/60, 1)
                }
            )
        else:
            continue
    all_options.sort(key = lambda x: x['price'])
    if not all_options:
        result = f"There is no Price available for all flights."
        return all_options, result 
    result = f"There are Flights from: {departure_id} to {arrival_id} on date: {travel_date}" 
    return all_options, result 


@tool
def One_way_flight_search(departure_id: str, arrival_id: str, travel_date: str):
    """
   Search for one-way flights between two airports on a specific date.
    Returns flight options with price, layovers, and duration details.
    
    Args:
        departure_id (str): Departure airport code (e.g., 'PEK' for Beijing)
        arrival_id (str): Arrival airport code (e.g., 'ISB' for Islamabad)
        travel_date (str): Travel date in YYYY-MM-DD format (e.g., '2026-03-03')
    
    Returns:
        str: Formatted flight search results with prices, durations, and layover information
    
    """
    params = {
    "engine": "google_flights",
    "departure_id": departure_id,
    "arrival_id": arrival_id,
    "currency": "USD",
    "type": "2",
    "outbound_date": travel_date,
    "api_key": ""
    }
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
    except Exception as e:
        return f"Failed to Fetch Flight Details: {e}"

    if results.get('error'):
        return f"There is some issue with the results: {results['error']}"
    lowest_available_price = results['price_insights']['lowest_price']
    current_price_status = results['price_insights']['price_level']
    average_price_status= results['price_insights']['typical_price_range']
    departure_airport = results['airports'][0]['departure'][0]['airport']['name']
    arrival_airport = results['airports'][0]['arrival'][0]['airport']['name']
    best_flight_result, best_flight_remarks = flight_info_extract(results.get("best_flights", []), departure_airport, arrival_airport, travel_date)
    other_flight_result, other_flight_remarks = flight_info_extract(results.get("other_flights", []), departure_airport, arrival_airport, travel_date)      
    return f"""
    Departure Airport {departure_airport}
    Arrival Airport {arrival_airport}
    Lowest Available Price: {lowest_available_price}
    Current Prices Status: {current_price_status}
    Average Price Status: {average_price_status}
    Best Flight Information:
        There are total of {len(best_flight_result)} flight options.
        Remarks on Best Flight: {best_flight_remarks}
        Overall Best Flight Info: {best_flight_result}
    Other Flight Information:
        There are total of {len(other_flight_result)} flight options.
        Remarks on Other Flights: {other_flight_remarks}
        Overall Other Flight Info: {other_flight_result}
    """


In [ ]:
from serpapi import GoogleSearch

params = {
  "engine": "google_flights",
  "departure_id": "CAN",
  "arrival_id": "ISB",
  "currency": "USD",
  "type": "2",
  "outbound_date": "2026-03-15",
  "api_key": ""
}

search = GoogleSearch(params)
results = search.get_dict()

In [ ]:
@tool
def attraction_finding_tool(place_name: str) -> list[dict]:
    """  
    Find popular attractions in a city or country.

    Args:
        place_name (str): Name of the city or country.

    Returns:
        list[dict]: List of up to 5 attractions with details:
            - title (str)
            - description (str)
            - rating (str/float)
            - highlighted_review (str)
        If no attractions found, returns a list with a dict containing an "error" key.
    """
    params = {
    "engine": "tripadvisor",
    "q": place_name,
    "ssrc": "A",
    "api_key": ""
    }

    search = GoogleSearch(params)
    try:

        results = search.get_dict()
        attraction_places = results.get("places", [])
    except Exception as e:
        return [{"error": f"There are no attraction places found in {place_name}."}]
    if not attraction_places:
        return []

    attraction_details = []

    for num in range(5):
        attraction = attraction_places[num]
        attraction_details.append(
            {
                "title": attraction.get('title', []),
                "description": attraction.get('description', []),
                "rating": attraction.get('rating', []),
                "highlighted_review": attraction.get('highlighted_review', [])
            }
        )
    return attraction_details
    


In [62]:
attraction_finding_tool.invoke({"place_name": "Islamabad"})

[{'title': 'Faisal Mosque',
  'description': 'Faisal Mosque is the mosque in Islamabad, Pakistan. Located on the foothills of Margalla Hills in Islamabad, the mosque features a contemporary design consisting of eight sides of concrete shell and is inspired by a Bedouin tent.',
  'rating': 4.6,
  'highlighted_review': {'text': 'Faisal Mosque is not just a mosque, it’s a symbol of Islamabad’s identity',
   'highlighted_texts': ['Islamabad’s'],
   'mention_count': 294}},
 {'title': 'The Centaurus',
  'description': [],
  'rating': 4.2,
  'highlighted_review': {'text': 'Overall:\r\n\r\nThe Centaurus Mall is a must-visit destination in Islamabad.',
   'highlighted_texts': ['Islamabad.'],
   'mention_count': 122}},
 {'title': 'Margalla Hills',
  'description': [],
  'rating': 4.6,
  'highlighted_review': {'text': 'Located in the northern side of Islamabad the capital.',
   'highlighted_texts': ['Islamabad'],
   'mention_count': 158}},
 {'title': 'Daman-e-Koh',
  'description': [],
  'rating'

In [ ]:
import os
from serpapi import GoogleSearch

@tool
def restaurant_finding_tool(place_name: str) -> list[dict]:
    """
    Tool Name: Restaurant Finder

    Purpose:
        Find top restaurants in a specified city using TripAdvisor.

    Input:
        place_name (str): Name of the city or country.

    Output:
        List of dictionaries (up to 5 items), each containing:
            - title (str): Name of the restaurant
            - description (str): Short description
            - rating (str/float): Average user rating
            - highlighted_review (str): Notable user review
        If no restaurants are found, returns a list with a dictionary containing an "error" key.

    Usage Instructions for AI:
        Use this tool when a user asks about restaurants or places to eat in a specific city.
    """
    
    params = {
        "engine": "tripadvisor",
        "q": place_name,
        "ssrc": "r",  # 'r' for restaurants
        "api_key": ""
    }

    search = GoogleSearch(params)

    try:
        results = search.get_dict()
        places = results.get("places", [])
    except Exception as e:
        return [{"error": f"Error fetching restaurants: {e}"}]

    if not places:
        return [{"error": f"No restaurants found in {place_name}."}]

    top_places = places[:5]

    restaurant_details = []
    for place in top_places:
        restaurant_details.append({
            "title": place.get("title", ""),
            "description": place.get("description", ""),
            "rating": place.get("rating", ""),
            "highlighted_review": place.get("highlighted_review", "")
        })

    return restaurant_details


In [ ]:
restaurant_finding_tool.invoke({"place_name": "Islamabad"})

[{'title': 'Kabul Restaurant',
  'description': 'Kabul restaurant started in 1983, and a good restaurant with a pleasing ambiance and quality and delicious food. Being one of the most prominent restaurants in F7 Markaz Islamabad provides you with a wide range of appetizers, and delicious Afghani food.',
  'rating': 4,
  'highlighted_review': {'text': 'I had the pleasure of dining at one of the best Restaurant in Islamabad',
   'highlighted_texts': ['Islamabad'],
   'mention_count': 82}},
 {'title': 'The Royal Elephant',
  'description': 'Offers Thai cusine. Two Private Rooms for Fine Dining one with the capacity of 10-persons and other with the capacity of 06-persons.',
  'rating': 4.3,
  'highlighted_review': {'text': 'We had a good time once again at the Royal Elephant at the Marriott Islamabad.',
   'highlighted_texts': ['Islamabad.'],
   'mention_count': 29}},
 {'title': 'chaayé khana',
  'description': 'The idea for Chaaye Khana was born from our love of tea. A cup of doodh patti 

In [ ]:
import os
from serpapi import GoogleSearch

@tool
def hotel_finding_tool(place_name: str) -> list[dict]:
    """
    Tool Name: Hotel Finder

    Purpose:
        Find top hotels in a specified city using TripAdvisor.

    Input:
        place_name (str): Name of the city or country.

    Output:
        List of dictionaries (up to 5 items), each containing:
            - title (str): Name of the hotel
            - description (str): Short description
            - rating (str/float): Average user rating
            - highlighted_review (str): Notable user review
        If no hotels are found, returns a list with a dictionary containing an "error" key.

    Usage Instructions for AI:
        Use this tool when a user asks about hotels, accommodations, or places to stay in a specific city.
    """
    
    params = {
        "engine": "tripadvisor",
        "q": place_name,
        "ssrc": "h",  # 'h' for hotels
        "api_key": ""
    }

    search = GoogleSearch(params)

    try:
        results = search.get_dict()
        places = results.get("places", [])
    except Exception as e:
        return [{"error": f"Error fetching hotels: {e}"}]

    if not places:
        return [{"error": f"No hotels found in {place_name}."}]

    top_places = places[:5]

    hotel_details = []
    for place in top_places:
        hotel_details.append({
            "title": place.get("title", ""),
            "description": place.get("description", ""),
            "rating": place.get("rating", ""),
            "highlighted_review": place.get("highlighted_review", "")
        })

    return hotel_details


In [ ]:
hotel_finding_tool.invoke({"city_name": "Islamabad"})

[{'title': 'Islamabad Serena Hotel',
  'description': "A visit to Islamabad Serena Hotel is more than just a comfortable stay at the only 5-star deluxe hotel in Pakistan, an affiliated member of the Leading Hotels of the World. It is a vivid immersion into the culture, heritage and lifestyles of Pakistan’s capital, a relatively undiscovered city boasting exhilarating scenery as well as some of the most colourful musical and culinary traditions in the world. Our Islamabad hotel is tucked into a peaceful, 14-acre garden setting between the Margalla Hills and Rawal Lake, framed by Islamic architecture and al fresco terraces and pavilions. Remaining faithful to traditional aesthetics with furniture and design flourishes handcrafted by local artisans, its timeless interiors have been curated to preserve a superb level of comfort. The experience is enhanced by outstanding Moorish architecture and ambience that revive old Islamic civilization and reflect the cultural heritage of Pakistan. Rel